## 1. Web Crawling for Stock Related News

## 2. Scrolling Down Page to Get More New

## 3. Text Mining

In [ ]:
#change artical content format for text mining
def artical_format(artical_content):
    artical_texts = []
    for i in artical_content:
        if len(i)>1:
            artical_texts.append([i[0]]+[''.join(i[1:])])
    return artical_texts

## 4.  Acquiring Stock Data

## 5. Data Visualization